# Introduction to Automation with LangChain, Generative AI, and Python
**1.1: Langchain**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

# 1.1: Introduction to LangChain

One of the most intriguing and promising developments in the evolving landscape of language models and artificial intelligence is LangChain. This technology represents a significant leap forward in how we interact with and harness the capabilities of large language models (LLMs). As we delve into the intricacies of LangChain in this chapter, it's important to understand not just the technical underpinnings but also the user experience that makes it so revolutionary.

## LangChain Chat Conversation Format

To explore LangChain comprehensively, we will adopt a format that has become increasingly familiar and effective in LLMs: the chat conversation interface. This interactive style, reminiscent of how many of us communicate daily, offers a unique and accessible means to illustrate LangChain's capabilities, potential applications, and the nuances of its operation.

We begin by importing the components from the LangChain library to support a chat-style interface to Bedrock. We will use the ChatBedrock interface for the Bedrock family of LLM models.

In [1]:
# Conversation Style Inteface

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

The conversation format consists of arrays of chat entries of the following three types:

* **SystemMessage** - This class designates the system prompt that provides instructions to the AI on the nature of the conversation and hints and guidelines. Generally, there will be only one system message at the beginning of the array.
* **HumanMessage** - This class designates the chat messages from outside the LLM, typically the human user.
* **AIMessage** - This class designates the chat messages from the LLM as responses to the HumanMessage messages.

Here we see the chain to ask a simple question.

In [2]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that concisely and accurately."
    ),
    HumanMessage(
        content="What is the capital of France?"
    ),
]

We now submit these messages and retrieve the output from the model. We will use the older and less expensive Titan model, which is good enough for this query. Further, we use a zero temperature; we are simply looking for a factual answer, and creativity is not a goal or concern.

In [3]:
from langchain_aws import ChatBedrock

MODEL = 'amazon.titan-text-lite-v1'
#MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'
#MODEL = 'mistral.mistral-7b-instruct-v0:2'

# Initialize bedrock, use built in role
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.1},
)

print("Model response:")
output = llm.invoke(messages)
print(output.content)
print("-----------")
print(output.response_metadata)

Model response:
 The capital of France is Paris.
-----------
{'model_id': 'amazon.titan-text-lite-v1', 'usage': {'prompt_tokens': 26, 'completion_tokens': 7, 'total_tokens': 33}}


The model that LangChain returns to you returns additional metadata. This data shows the token usage, which might be useful for estimating the total cost expected from this query.

We can continue to grow this conversation if we wish. To do so, we added the model's response and another human question. Here, we will ask the model if it was sure about its last response.

In [4]:
messages.append(output)
messages.append(HumanMessage(content="Are you sure, I think it was renamed for some reason?"))
for message in messages:
    print(f"{type(message).__name__} : {message.content}")

SystemMessage : You are a helpful assistant that concisely and accurately.
HumanMessage : What is the capital of France?
AIMessage :  The capital of France is Paris.
HumanMessage : Are you sure, I think it was renamed for some reason?


We can submit the conversation array to the model and see its latest response.

In [5]:
print("Model response:")
output = llm.invoke(messages)
print(output.content)

Model response:
 Paris is the capital of France. It has been the capital since 1792.


## Asking a Single Question

If you wish to ask the model a single question, not as part of a conversation chain, you can pass a string to the model for a response.

In [6]:
# Define the question
question = "What are the five largest cities in the USA by population?"

# Use Langchain to call the API
# The method and parameters might differ based on the Langchain version
response = llm.invoke(question)

# Print the response
display(response.content)

' The five largest cities in the USA by population are: New York City, Los Angeles, Chicago, Houston, and Phoenix.'

## Prompt Templates

LangChain allows you to create chains of operations typically performed as part of an LLM-enabled application. One of these operations is a prompt template, which allows you to insert text into a previously created prompt. In this example, we will create a prompt template that asks the model to create a random blog post title.

```
Return only the title of a blog post article title on the topic of {topic} in {language}
```

To accomplish this objective, we will use a **PromptTemplate** object.

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Higher temperature, more creative
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.9},
)

topic = "pets for data scientists"
language = "english"

title_template = PromptTemplate( input_variables = ['topic', 'language'],\
  template = 'Return only the title of a blog post article title on the topic of {topic} in {language}' )
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True)
response = title_chain.invoke({'topic':topic,'language':language })
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Return only the title of a blog post article title on the topic of pets for data scientists in english

> Finished chain.
{'topic': 'pets for data scientists', 'language': 'english', 'text': ' You can find the title of a blog post article by using the following query in English:\n\n```\n"title": {"$regex": "\\\\b(pet|dogs|cats|animals|pet food|pet shop|pet health|pet grooming|pet accessories|pet supplies|pet training|pet lifestyle|pet adoption|pet shop online|pet names|pet grooming tips|pet health tips|pet photos|pet care tips|pet tricks|pet training tricks|pet lover|pet blog|pet care advice|pet nutrition|pet shop advice|pet shop tips|pet advice|pet tips|pet life|'}


## Create a Simple Sequential Chain

We will now use LangChain to tie multiple LLM calls into a longer chain using the **SimpleSequentialChain** class. We will use two smaller chains to create a title and body text for a blog post. We begin by defining the two prompts we will use to construct this blog post. Also, note that we request that the LLM utilize [markdown](https://en.wikipedia.org/wiki/Markdown) to generate the actual blog post.


In [8]:
# Create the two prompt templates
title_template = PromptTemplate( input_variables = ['topic'], template = 'Give me a blog post title on {topic} in English' )
article_template = PromptTemplate( input_variables = ['title'], template = 'Write a blog post for {title}, format in markdown.' )

We will create the first chain to generate the random title. Here, we allow the user to specify the topic. We use a higher temperature to increase the creativity of the title. We also use a simpler model to minimize cost for the relatively simple task of title selection.

In [9]:
MODEL = 'amazon.titan-text-lite-v1'

# Higher temperature, more creative
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.9},
)
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True)

Next, we compose the actual blog post; we will use a lower temperature to decrease creativity and cause the LLM to stick to factual information and avoid hallucinations. We also use a more complex model to provide a better article.

In [10]:
MODEL2 = 'anthropic.claude-3-sonnet-20240229-v1:0'

# Create the article chain, stronger mode, still creative
llm2 = ChatBedrock(
    model_id=MODEL2,
    model_kwargs={"temperature": 0.9},
)
article_chain = LLMChain(llm=llm2, prompt=article_template, verbose=True)

Now, we combine these two chains into one. The input to the first chain will be the selected topic. The first chain will then output the title to the second chain, which will, in turn, output the actual article.

In [11]:
# Create a complete chain to create a new blog post
complete_chain=SimpleSequentialChain(chains=[title_chain, article_chain], verbose=True)

We can now display the final article. In this case, we requested an article on "photography," and displayed the final article's markdown.

In [12]:
from IPython.display import display_markdown

article = complete_chain.invoke('photography')



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Give me a blog post title on photography in English

> Finished chain.
 Sure, I can help you with that. Here's a blog post title on photography in English:

"How to Take Professional-Looking Photos with Your Phone: A Beginner's Guide"


> Entering new LLMChain chain...
Prompt after formatting:
Write a blog post for  Sure, I can help you with that. Here's a blog post title on photography in English:

"How to Take Professional-Looking Photos with Your Phone: A Beginner's Guide", format in markdown.

> Finished chain.
# How to Take Professional-Looking Photos with Your Phone: A Beginner's Guide

In today's world, where smartphones have become an integral part of our daily lives, the art of photography has become more accessible than ever before. With the advancements in mobile camera technology, it's now possible to capture stunning, professional-looking photos right from your phone

The actual display of the markdown is handled by this code:

In [13]:
output_key = complete_chain.output_key

display_markdown(article[output_key], raw=True)

# How to Take Professional-Looking Photos with Your Phone: A Beginner's Guide

In today's world, where smartphones have become an integral part of our daily lives, the art of photography has become more accessible than ever before. With the advancements in mobile camera technology, it's now possible to capture stunning, professional-looking photos right from your phone. Whether you're an aspiring photographer or simply someone who enjoys capturing memorable moments, this beginner's guide will help you elevate your mobile photography game.

## Master the Basics

Before diving into advanced techniques, it's essential to understand the fundamentals of photography. Start by learning about the exposure triangle, which consists of aperture, shutter speed, and ISO. While most smartphones don't allow you to manually adjust these settings, understanding how they work will help you make informed decisions when using your phone's camera.

## Explore Your Phone's Camera Features

Most modern smartphones come equipped with a variety of camera features and modes. Take some time to explore these features and experiment with different settings. For example, try using the portrait mode to achieve a beautiful bokeh effect (blurred background) or the panorama mode to capture stunning landscapes.

## Compose Your Shots

Composition is a crucial element in photography, and it can make or break a photo. Follow the rule of thirds, which suggests dividing your frame into thirds horizontally and vertically, and positioning your subject along these lines or at their intersections. Additionally, consider leading lines, symmetry, and framing to create visually appealing compositions.

## Take Advantage of Natural Light

Natural light is one of the most beautiful and versatile light sources for photography. Whenever possible, try to shoot in natural light conditions, such as during the golden hours (shortly after sunrise or before sunset) or on overcast days, which provide soft, diffused light. If shooting indoors, position your subject near a window or use reflectors to bounce light onto your subject.

## Edit and Enhance Your Photos

While it's important to capture great shots in-camera, post-processing can take your photos to the next level. Explore the editing tools available on your phone or download third-party photo editing apps. These apps can help you adjust exposure, contrast, saturation, and more, allowing you to bring out the best in your photographs.

## Practice, Practice, Practice

Like any skill, photography requires practice. The more you shoot, the more you'll develop an eye for composition, lighting, and other essential elements. Carry your phone with you wherever you go and capture everything that catches your eye. Experiment with different angles, perspectives, and settings to find your unique style.

Remember, taking professional-looking photos with your phone is a journey, and with dedication and practice, you'll be amazed at the stunning results you can achieve. So, grab your phone, embrace your creativity, and start capturing the world around you in a whole new light!